In [1]:
# System and processing
import sys
import dill as pickle
import copy
from tqdm import tqdm
import os

from multiprocessing import Process, Queue, Pool
import multiprocessing
from pathos.multiprocessing import ProcessingPool, ThreadingPool
from multiprocessing import Manager

import itertools
from itertools import permutations, combinations


# Common math packages
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import random
import pandas as pd
from numpy.linalg import norm

# External packages
import sage.all
import sage.geometry.polyhedron.base as Polyhedron

# Codes
import params_team
from simple_rl.agents import FixedPolicyAgent
from simple_rl.planning import ValueIteration
from simple_rl.utils import make_mdp
from policy_summarization import bayesian_IRL
from policy_summarization import policy_summarization_helpers as ps_helpers
from policy_summarization import BEC
import policy_summarization.multiprocessing_helpers as mp_helpers
from simple_rl.utils import mdp_helpers
import policy_summarization.BEC_helpers as BEC_helpers
import policy_summarization.BEC_visualization as BEC_viz
from teams import particle_filter_team as pf_team
import teams.teams_helpers as team_helpers
import simulation.sim_helpers as sim_helpers
import teams.utils_teams as utils_teams
from analyze_sim_data import run_analysis_script


# Plotting
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
from termcolor import colored
matplotlib.use('TkAgg')

mpl.rcParams['figure.facecolor'] = '1.0'
mpl.rcParams['axes.labelsize'] = 'x-large'
mpl.rcParams['xtick.labelsize'] = 'large'
plt.rcParams['figure.figsize'] = [15, 10]

from statsmodels.graphics.factorplots import interaction_plot

# Other imports.
sys.path.append("simple_rl")



/home/sureshkj/anaconda3/envs/aug_taxi/lib/python3.9/site-packages/tensorflow/python/framework/dtypes.py:246: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),


In [2]:
def get_optimal_policies(params, pool, lock):
    
    """
    Gets the minimum BEC constranints for the domain. Need to load into the database.
    """

    ps_helpers.obtain_env_policies(params.mdp_class, params.data_loc['BEC'], np.expand_dims(params.weights['val'], axis=0), params.mdp_parameters, pool, lock)

    # get base constraints for all the environments and demonstrations
    try:
        with lock:
            with open('models/' + params.data_loc['BEC'] + '/team_base_constraints.pickle', 'rb') as f:
                policy_constraints, min_subset_constraints_record, env_record, traj_record, traj_features_record, reward_record, mdp_features_record, consistent_state_count = pickle.load(f)
    except:
        # use policy BEC to extract constraints
        policy_constraints, min_subset_constraints_record, env_record, traj_record, traj_features_record, reward_record, mdp_features_record, consistent_state_count = BEC.extract_constraints(params.data_loc['BEC'], params.BEC['BEC_depth'], params.step_cost_flag, pool, lock, print_flag=True)
        with lock:
            with open('models/' + params.data_loc['BEC'] + '/team_base_constraints.pickle', 'wb') as f:
                pickle.dump((policy_constraints, min_subset_constraints_record, env_record, traj_record, traj_features_record, reward_record, mdp_features_record, consistent_state_count), f)

    # get BEC constraints
    try:
        with lock:
            with open('models/' + params.data_loc['BEC'] + '/team_BEC_constraints.pickle', 'rb') as f:
                min_BEC_constraints, BEC_lengths_record = pickle.load(f)
    except:
        min_BEC_constraints, BEC_lengths_record = BEC.extract_BEC_constraints(policy_constraints, min_subset_constraints_record, env_record, params.weights['val'], params.step_cost_flag, pool)
        with lock:
            with open('models/' + params.data_loc['BEC'] + '/team_BEC_constraints.pickle', 'wb') as f:
                pickle.dump((min_BEC_constraints, BEC_lengths_record), f)
        
    return min_subset_constraints_record, env_record, traj_record, traj_features_record, mdp_features_record, consistent_state_count, min_BEC_constraints


In [3]:
def init_pool_processes(the_lock):
    '''Initialize each process with a global variable lock.
    '''
    global file_lock
    lock = the_lock


class NoDaemonProcess(multiprocessing.Process):
    
    def __init__(self, lock, *args, **kwargs):
        self._lock = lock
        super().__init__(*args, **kwargs)
    
    
    # make 'daemon' attribute always return False
    @property
    def daemon(self):
        return False

    @daemon.setter
    def daemon(self, val):
        pass

    @property
    def lock(self):
        return self._lock
    
    def set_lock(self, lock):
        self._lock = lock

    def run(self):
        if self._lock:
            init_pool_processes(self._lock)
        super().run()


class NoDaemonProcessPool(multiprocessing.pool.Pool):

    def __init__(self, processes=None, lock=None, *args, **kwargs):
        self._lock = lock
        super().__init__(processes=processes, initializer=self.init_pool_processes, *args, **kwargs)

    def init_pool_processes(self):
        global lock
        lock = self._lock

    def Process(self, *args, **kwds):
        proc = super(NoDaemonProcessPool, self).Process(*args, **kwds)
        proc.__class__ = NoDaemonProcess
        proc._lock = self._lock  # Pass the lock to the process

        return proc

In [18]:
def initialize_teaching(params, teacher_learning_factor, pool, lock):
    
    min_subset_constraints_record, env_record, traj_record, traj_features_record, mdp_features_record, consistent_state_count, min_BEC_constraints = get_optimal_policies(params, pool, lock)

    
    team_prior, particles_team_teacher = team_helpers.sample_team_pf(params.team_size, params.BEC['n_particles'], params.weights['val'], params.step_cost_flag, teacher_learning_factor=teacher_learning_factor, team_prior = params.team_prior, model_type = params.teacher_update_model_type)
    
    variable_filter, nonzero_counter, teaching_complete_flag = team_helpers.check_and_update_variable_filter(min_subset_constraints_record, initialize_filter_flag=True)
    
    visited_env_traj_idxs, min_BEC_constraints_running = [], copy.deepcopy(params.prior)
    
    return particles_team_teacher,  variable_filter, nonzero_counter, min_BEC_constraints_running, visited_env_traj_idxs

In [33]:
def run_teaching_loop(params, args):
    
    loop_count = 1
    
    particles_demo, summary_pool, lock, vars_filename, variable_filter, min_BEC_constraints_running, visited_env_traj_idxs = args
    
    # fixed variables (maybe better to read from file)
    min_subset_constraints_record, env_record, traj_record, traj_features_record, mdp_features_record, consistent_state_count, min_BEC_constraints = get_optimal_policies(params, summary_pool, lock)
    teacher_uf_demo = 0.8
    
    # check if learning is complete
    
    
    # generate demos
    demonstration_summary, _, min_BEC_constraints_running, visited_env_traj_idxs, particles_demo = team_helpers.obtain_team_summary(params, params.data_loc['BEC'], vars_filename, min_subset_constraints_record, min_BEC_constraints, env_record, traj_record, mdp_features_record, params.weights['val'], params.step_cost_flag, 
                                                                                                            summary_pool, lock, params.BEC['n_human_models'], consistent_state_count, particles_demo, teacher_uf_demo, 'p1', 
                                                                                                            variable_filter, [], [], 0, min_BEC_constraints_running, visited_env_traj_idxs, vars_filename = vars_filename)        
      
    print('demonstration_summary: ', demonstration_summary)
    
    if len(demonstration_summary) == 0:
        print('No new demonstrations possible!')
        repeat_prev_demo_flag = True
    else:
        repeat_prev_demo_flag = False
        
    demonstration = demonstration_summary[-1]
    # obtain diagnostic tests
    unit_constraints, demo_ids, running_variable_filter_unit = team_helpers.show_demonstrations(demonstration, particles_demo, params.mdp_class, params.weights['val'], loop_count, viz_flag = True)
    min_KC_constraints = BEC_helpers.remove_redundant_constraints(unit_constraints, params.weights['val'], params.step_cost_flag)
    preliminary_tests, visited_env_traj_idxs = team_helpers.obtain_diagnostic_tests(lock, params.data_loc['BEC'], demonstration, visited_env_traj_idxs, min_KC_constraints, min_subset_constraints_record, traj_record, traj_features_record, running_variable_filter_unit, mdp_features_record)

    demo_mdps = []
    for demo in demonstration:
        demo_mdps.append(demo[0])  #Demo MDP
        
    test_mdps = []
    for test in preliminary_tests:
        test_mdps.append([test[0], test[1]]) #Test MDP and Optimal Trajectory
        
        
    return demo_mpds, test_mdps, variable_filter, min_BEC_constraints_running, visited_env_traj_idxs
    

In [ ]:
params = copy.deepcopy(params_team)

teacher_learning_factor = [0.8, 0.8, 0.8]

vars_filename = 'test_run'

with Manager() as manager:
    lock = manager.Lock()
    
    full_path_filename = 'models/' + params.data_loc['BEC'] + '/ind_sim_trials/' + vars_filename

    # create a folder for this run
    if not os.path.exists(full_path_filename):
        with lock:
            print(colored('Creating folder for this run: ', 'yellow'), full_path_filename)
            os.makedirs(full_path_filename, exist_ok=True)
    
    with lock:
         open('models/' +  params.data_loc['BEC']  + '/ind_sim_trials/' + vars_filename + '/demo_gen_log.txt', 'w').close()

    summary_pool = Pool(min(params.n_cpu, 60), initializer=init_pool_processes, initargs=(lock,))

    particles_team_teacher, variable_filter, nonzero_counter, min_BEC_constraints_running, visited_env_traj_idxs  = initialize_teaching(params, teacher_learning_factor,  summary_pool, lock)

    particles_demo = copy.deepcopy(particles_team_teacher['p1'])

    args = particles_demo, summary_pool, lock, vars_filename, variable_filter, min_BEC_constraints_running, visited_env_traj_idxs

    demo_mpds, test_mdps, variable_filter, min_BEC_constraints_running, visited_env_traj_idxs = run_teaching_loop(params, args)

Solving for the optimal policy in each environment:


0it [00:00, ?it/s]


Initializing particle filter..
num_points:  1000
Updating particles for  p1 with prior knowledge in  noise  condition...


/home/sureshkj/anaconda3/envs/aug_taxi/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:178: RuntimeWarning: The number of calls to function has reached maxfev = 400.
  warnings.warn(msg, RuntimeWarning)


Resampling for constraint:  [[ 0  0 -1]]
cumulative_sum weights end:  0.9999999999999976 positions end:  0.999353218289981
Take the largest gap/azimuth distance between two consecutive particles.
Take the largest gap/elevation distance between two consecutive particles.
Initializing particle filter..
num_points:  1000
Updating particles for  p2 with prior knowledge in  noise  condition...
Resampling for constraint:  [[ 0  0 -1]]
cumulative_sum weights end:  0.9999999999999976 positions end:  0.9993889101503945
Take the largest gap/azimuth distance between two consecutive particles.
Take the largest gap/elevation distance between two consecutive particles.
Initializing particle filter..
num_points:  1000
Updating particles for  p3 with prior knowledge in  noise  condition...
Resampling for constraint:  [[ 0  0 -1]]
cumulative_sum weights end:  0.9999999999999976 positions end:  0.9996159470387984
Take the largest gap/azimuth distance between two consecutive particles.
Take the largest g

...

Opening models/augmented_taxi2/counterfactual_data_precomputed/model1380/cf_data_env00036.pickle
Opening models/augmented_taxi2/counterfactual_data_precomputed/model1380/cf_data_env00038.pickle
Opening models/augmented_taxi2/counterfactual_data_precomputed/model2466/cf_data_env00048.pickle
Opening models/augmented_taxi2/counterfactual_data_precomputed/model1380/cf_data_env00040.pickle
Opening models/augmented_taxi2/counterfactual_data_precomputed/model1380/cf_data_env00042.pickle


Opening models/augmented_taxi2/counterfactual_data_precomputed/model1380/cf_data_env00044.pickle
Opening models/augmented_taxi2/counterfactual_data_precomputed/model1380/cf_data_env00046.pickle
Opening models/augmented_taxi2/counterfactual_data_precomputed/model2466/cf_data_env00050.pickle
Opening models/augmented_taxi2/counterfactual_data_precomputed/model2466/cf_data_env00052.pickle
Opening models/augmented_taxi2/counterfactual_data_precomputed/model2466/cf_data_env00054.pickle
Opening models/augmented_taxi2/counterfactual_data_precomputed/model2466/cf_data_env00056.pickle
Opening models/augmented_taxi2/counterfactual_data_precomputed/model2466/cf_data_env00058.pickle
Opening models/augmented_taxi2/counterfactual_data_precomputed/model2466/cf_data_env00060.pickle
Opening models/augmented_taxi2/counterfactual_data_precomputed/model2466/cf_data_env00062.pickle
Opening models/augmented_taxi2/counterfactual_data_precomputed/model1297/cf_data_env00048.pickle
Opening models/augmented_taxi2

Opening models/augmented_taxi2/counterfactual_data_precomputed/model1826/cf_data_env00054.pickle
Opening models/augmented_taxi2/counterfactual_data_precomputed/model1826/cf_data_env00056.pickle
Opening models/augmented_taxi2/counterfactual_data_precomputed/model1826/cf_data_env00058.pickle
Opening models/augmented_taxi2/counterfactual_data_precomputed/model1826/cf_data_env00060.pickle
Opening models/augmented_taxi2/counterfactual_data_precomputed/model1826/cf_data_env00062.pickle
Opening models/augmented_taxi2/counterfactual_data_precomputed/model1283/cf_data_env00048.pickle
Opening models/augmented_taxi2/counterfactual_data_precomputed/model1283/cf_data_env00050.pickle
Opening models/augmented_taxi2/counterfactual_data_precomputed/model1283/cf_data_env00052.pickle
Opening models/augmented_taxi2/counterfactual_data_precomputed/model1283/cf_data_env00054.pickle
Opening models/augmented_taxi2/counterfactual_data_precomputed/model1283/cf_data_env00056.pickle
Opening models/augmented_taxi2

Opening models/augmented_taxi2/counterfactual_data_precomputed/model1383/cf_data_env00062.pickle
Opening models/augmented_taxi2/counterfactual_data_precomputed/model2366/cf_data_env00048.pickle
Opening models/augmented_taxi2/counterfactual_data_precomputed/model2366/cf_data_env00050.pickle
Opening models/augmented_taxi2/counterfactual_data_precomputed/model2366/cf_data_env00052.pickle
Opening models/augmented_taxi2/counterfactual_data_precomputed/model2366/cf_data_env00054.pickle
Opening models/augmented_taxi2/counterfactual_data_precomputed/model2366/cf_data_env00056.pickle
Opening models/augmented_taxi2/counterfactual_data_precomputed/model2366/cf_data_env00058.pickle
Opening models/augmented_taxi2/counterfactual_data_precomputed/model2366/cf_data_env00060.pickle
Opening models/augmented_taxi2/counterfactual_data_precomputed/model2366/cf_data_env00062.pickle
Opening models/augmented_taxi2/counterfactual_data_precomputed/model1962/cf_data_env00048.pickle
Opening models/augmented_taxi2

/home/sureshkj/anaconda3/envs/aug_taxi/lib/python3.9/site-packages/scipy/spatial/_geometric_slerp.py:220: UserWarning: start and end are antipodes using the specified tolerance; this may cause ambiguous slerp paths
  warnings.warn("start and end are antipodes"


In [ ]:
print (d)